In [ ]:
pip install flexvalue

In [ ]:
!flexvalue download-avoided-costs-data-db --year 2020

In [ ]:
import pandas as pd

In [ ]:
try:
    from google.colab import files
    colab = True
except:
    colab = False

In [ ]:
if colab:

    user_inputs_dict = files.upload()

    if len(user_inputs_dict)!=1:
        print("Must select exactly one file")
    else:
        user_input_filename = list(user_inputs_dict.keys())[0]
        user_inputs = pd.read_csv(user_input_filename)

In [ ]:
if colab:
    metered_ls_dict = files.upload()
    if len(metered_ls_dict)==0:
        metered_load_shape = None
    elif len(metered_ls_dict) == 1:
        metered_load_shape = list(metered_ls_dict.values())[0]
    else:
        print("Must select exactly one file")

In [ ]:
from flexvalue import FlexValueRun

year = '2020'
flx_run = FlexValueRun(database_year=year, metered_load_shape=metered_load_shape)

outputs_table, electric_benefits, gas_benefits = flx_run.get_results(user_inputs=user_inputs)

from pathlib import Path
user_input_filename_no_ext = Path(user_input_filename).stem
outputs_filename = f'{user_input_filename_no_ext}_outputs.csv'
outputs_table.to_csv(outputs_filename)

In [ ]:
from IPython.display import FileLink
display(FileLink(outputs_filename))
display(outputs_table)

In [ ]:
from flexvalue.plots import plot_results
plot_results(outputs_table, electric_benefits, gas_benefits)